In [22]:
import webdriver_manager.chrome
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from pynput.keyboard import Key, Controller
import re
import sqlite3

keyboard = Controller()
conn = sqlite3.connect('data.db')
c = conn.cursor()

# Create companies table
c.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='companies' ''')
if c.fetchone()[0] == 0:
    c.execute('''CREATE TABLE companies(
            id integer NOT NULL PRIMARY KEY AUTOINCREMENT,
            name varchar NOT NULL DEFAULT ''
        )
    ''')
    c.execute('''CREATE UNIQUE INDEX companies_unique_name ON companies(name)''')
    c.execute('''INSERT INTO companies (id, name) VALUES (0, '') ''')
else:
    print("Table companies exists")

# Create technologies table
c.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='techs' ''')
if c.fetchone()[0] == 0:
    c.execute('''CREATE TABLE techs(
            id varchar NOT NULL PRIMARY KEY,
            regex varchar NOT NULL
        )
    ''')
    c.execute('''CREATE UNIQUE INDEX techs_unique_name ON techs(id)''')
    c.execute('''CREATE UNIQUE INDEX techs_unique_regex ON techs(regex)''')
else:
    print("Table techs exists")

# Create positions table
c.execute(''' SELECT count(name) FROM sqlite_master WHERE type='table' AND name='positions' ''')
if c.fetchone()[0] == 0:
    c.execute('''CREATE TABLE positions(
            id integer NOT NULL PRIMARY KEY AUTOINCREMENT,
            name varchar NOT NULL,
            salary real NULL,
            company_id integer NOT NULL DEFAULT 0,
            tech_id varchar NOT NULL,
            source varchar NOT NULL,
            FOREIGN KEY(company_id) REFERENCES companies(id),
            FOREIGN KEY(tech_id) REFERENCES techs(id)
        )
    ''')
else:
    print("Table positions exists")

# Add Programming Languages
c.execute('''INSERT OR IGNORE INTO techs (id, regex) VALUES
('Javascript', 'javascript'),
('Java', '[ ,]java[ \.,]'),
('C', '[ ,]c[ \.,]'),
('C++', 'c\+\+'),
('Kotlin', 'kotlin'),
('Swift', 'swift'),
('Ruby', 'ruby'),
('Python', 'python3?'),
('SQL', 'sql'),
('R', '[ ,]r[ \.,]'),
('Perl', 'perl'),
('C#', 'c#'),
('Lua', 'lua'),
('Rust', 'rust'),
('PHP', 'php'),
('HTML', 'html'),
('CSS', 'css'),
('XML', 'xml')
''')

conn.commit()

Table companies exists
Table techs exists


In [140]:
# Code for data collection
from bs4 import BeautifulSoup as bs
import requests as rq

langs = {"javascript": [], " c[ \.]": [], "c\+\+": [], "java": [], "kotlin": [], "swift": [], "ruby": [], "python3?": [], "sql": [], " r[ \.]": [], "perl": [], "c#": [], "lua": [], "rust": [], "php": [], "html": [], "css": [], "xml": []}

page = rq.get("https://www.linkedin.com/jobs/search?keywords=Software%2Bengineer&location=United%20States&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0")
soup = bs(page.content, 'html.parser')
items = soup.find_all(class_="base-card__full-link")
print(langs.values())
salary_match = "\$?\d{0,3},?\d{1,3},\d{3,3}\.?\d{0,2}"
item = items[0]
for item in items:
    url = item.get('href')
    print(url + "\n")
    posting_page = rq.get(url)
    s = bs(posting_page.content, 'html.parser')
    text = s.get_text()
    sal_matches = list(map(lambda x: float(re.sub("[^0-9\.]", "", x)), re.findall(salary_match, text)))
    for k, v in langs.items(): 
        if re.search(k, text, flags=re.IGNORECASE) != None:
            langs[k].extend(sal_matches)
    
print(langs)

dict_values([[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []])
https://www.linkedin.com/jobs/view/c%2B%2B-software-engineer-at-jobot-3034297216?refId=XKzjLKYR%2FR2gZADddIXHkQ%3D%3D&trackingId=FxKZ5GltoHktaVtQRXH%2ByA%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card

https://www.linkedin.com/jobs/view/software-engineer-c%23-at-itw-3039563164?refId=XKzjLKYR%2FR2gZADddIXHkQ%3D%3D&trackingId=xtJXCmxyuGtI7%2BKZDHbmsw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card

https://www.linkedin.com/jobs/view/%232833-software-engineer-react-native-at-it-motives-3029776271?refId=XKzjLKYR%2FR2gZADddIXHkQ%3D%3D&trackingId=t1z6oqJDI65vv03pGROiTA%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card

https://www.linkedin.com/jobs/view/backend-software-engineer-data-ingestion-at-gridium-3042407691?refId=XKzjLKYR%2FR2gZADddIXHkQ%3D%3D&trackingId=lJp5KTl7jLJaBKFVYi5DOw%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-ca

In [21]:
# import csv

langs = {"javascript": [], " c[ \.]": [], "c\+\+": [], "java": [], "kotlin": [], "swift": [], "ruby": [], "python3?": [], "sql": [], " r[ \.]": [], "perl": [], "c#": [], "lua": [], "rust": [], "php": [], "html": [], "css": [], "xml": []}

scraped_jobs = []
job_set = set()

# Initialize Driver
url = 'https://www.glassdoor.com/Job/software-engineer-jobs-SRCH_KO0,17.htm'
driver = webdriver.Chrome(webdriver_manager.chrome.ChromeDriverManager().install())
driver.get(url)

# WHEN SELENIUM IS RUNNING, DON'T CLICK AROUND OTHER TABS, THIS WILL BREAK PROGRAM
def main():
    keyboard.press(Key.esc)
    keyboard.release(Key.esc)
    
    num_pages_to_search = 2
    num_pages_searched = 0
    num_jobs_searched = 0
    
    # extract the job data in "num_pages_to_search" pages
    while num_pages_searched < num_pages_to_search:
        # Each job
        jobs_on_page = driver.find_elements_by_class_name('react-job-listing')
        
        for job_card in jobs_on_page:
            # sleep to load the info 
            sleep(0.75)
            keyboard.press(Key.esc)
            keyboard.release(Key.esc)
            keyboard.press(Key.esc)
            keyboard.release(Key.esc)
            
            try:
                job_description = driver.find_element_by_class_name('jobDescriptionContent').text.lower() # THIS IS JOB DESCRIPTION
                languages = extract_languages(job_description) # TODO: Extract Programming Languages from Job Description
                salary = process_salary(driver.find_element_by_class_name('css-y2jiyn').text)
                company = process_company(driver.find_element_by_class_name('css-xuk5ye').text)
                job_position = driver.find_element_by_class_name('css-1j389vi').text
                format_data((company, job_position, salary)) # Format information
            except NoSuchElementException:
                print("No Salary")

            job_card.click()

        sleep(1)
        try:
            text = driver.find_element_by_class_name('jobDescriptionContent').text.lower()
            salary = process_salary(driver.find_element_by_class_name('css-y2jiyn').text)
            company = process_company(driver.find_element_by_class_name('css-xuk5ye').text)
            job_position = driver.find_element_by_class_name('css-1j389vi').text
            format_data((company, job_position, salary))
        except NoSuchElementException:
            print("No Salary")
        num_pages_searched += 1
        if num_pages_searched == num_pages_to_search:
            print("Finished Searching. Check the salaries.csv file for output")
        
        # go to next page
        driver.find_element_by_class_name('nextButton').click()
        sleep(5)
    record_data(scraped_jobs)

# Helper function to extract programming languages from the job description text
def extract_languages(job_description):
    return
    
# Used to get the name of Job company without star ratings
def process_company(company):
    try:
        if driver.find_element_by_class_name('css-1m5m32b') != None:
            return company[0:-4]
    except NoSuchElementException:
        return company

# $70,000 /yr (est.) => 70000
# $35 /hr (est.) => 35*52*40
def process_salary(salary):
    new_salary = salary.split(' ', 1)
    if new_salary[1] == '/yr (est.)':
        return new_salary[0][1:]
    else:
        return str(float(new_salary[0][1:]) * 52 * 40)
        
# Format the data received from parsing a Job Card
def format_data(job_row):
    if job_row not in job_set:
        print(job_row)
        scraped_jobs.append(job_row)
        job_set.add(job_row)

# At the end, I want to record all my unique jobs to a CSV file
def record_data(records):
    with open('salaries.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Company','Job Title', 'Salary'])
        writer.writerows(records)
        
# Run my main program
main()




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/christiantingle/.wdm/drivers/chromedriver/mac64_m1/101.0.4951.41/chromedriver] found in cache
/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_77338/1600393057.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(webdriver_manager.chrome.ChromeDriverManager().install())
/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_77338/1600393057.py:25: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  jobs_on_page = driver.find_elements_by_class_name('react-job-listing')
/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_77338/1600393057.py:36: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  job

No Salary
No Salary


/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_77338/1600393057.py:39: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  company = process_company(driver.find_element_by_class_name('css-xuk5ye').text)
/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_77338/1600393057.py:72: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  if driver.find_element_by_class_name('css-1m5m32b') != None:
/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_77338/1600393057.py:40: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  job_position = driver.find_element_by_class_name('css-1j389vi').text


('Oracle', 'Software Developer 5 2022P', '145,517')
('KUBRA', 'Java Software Engineer', '112,665')
('Psychology Today', 'Software Engineer (Remote)', '160,000')
('KOHLS', 'Software Engineer', '102,206')
No Salary
('BOEING', 'Entry-Level Software Engineer for Embedded Training', '92,926')
('Renaissance', 'Software Engineer III', '106,500')
('KOHLS', 'Software Security Engineer - Identity (Remote Opportunity)', '77,762')
('KOHLS', 'Software Engineers - Full Stack and Backend (REMOTE)', '82,969')
('ATR International, Inc.', 'Jr. Software Engineer', '86320.0')
('National Security Agency', 'Software Engineer', '100,725')
('KOHLS', 'Platform Engineer - Middleware - REMOTE Opportunity', '83,681')
('MIM Software', 'Software Services Engineer', '195000.0')
('Liberty Mutual Insurance', 'Software Engineer - Software Delivery (Remote)', '98,025')
('Confidential', 'Senior Software Engineer', '110,000')
('Plaudit Design', 'Full-Stack Programmer (Remote or St Paul Office)', '54,254')
('HP', 'Entry Le

/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_77338/1600393057.py:49: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  text = driver.find_element_by_class_name('jobDescriptionContent').text.lower()
/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_77338/1600393057.py:50: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  salary = process_salary(driver.find_element_by_class_name('css-y2jiyn').text)
/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_77338/1600393057.py:51: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  company = process_company(driver.find_element_by_class_name('css-xuk5ye').text)
/var/folders/2d/xzxff1d56t5081dv50t49vj80000gn/T/ipykernel_77338/1600393057.py:52: DeprecationWarning: find_element_by_class_name is dep

('Liberty Mutual Insurance', 'Senior Software Engineer', '129,151')
('Liberty Mutual Insurance', 'Solutions Engineer, Software', '167,901')
('Wicresoft', 'Software Engineer for Microsoft Defender', '95,000')
('Icahn School of Medicine at Mount Sinai', 'Software Engineer', '118,146')
('Revature', 'Entry Level Software Engineer Virtual Hiring Event', '83,289')
('Liberty Mutual Insurance', 'Software Engineer - Remote', '107,050')
No Salary
('HP', 'Entry Level Full Stack Software Engineer (back-end focus)”', '91,524')
No Salary
('Vision Government Solutions', 'Senior Backend Software Engineer (Remote)', '135,000')
No Salary
('Vision Government Solutions', 'Backend Software Engineer (Remote)', '105,000')
('Indeed', 'Principal Software Engineer - SMB', '205,000')
('Indeed', 'Fullstack Software Engineer - Enterprise', '110,000')
('National Security Agency', 'Hardware/Software Design Engineer', '101,589')
('Next Step Systems', 'Software Platform Operations Engineer, SaaS and Oracle - Work From

NameError: name 'csv' is not defined